# 1. 크롤링

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [3]:
import pickle

In [28]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [5]:
isbn13 = pd.read_pickle('isbn13.pkl')

In [194]:
isbn13

0        9791190710343
1        9791190710350
2        9791190710367
3        9791128858338
4        9788954685603
             ...      
54971    9791185020136
54972    9788997838127
54973    9788956591858
54974    9788987794938
54975    9788995696972
Name: isbn13, Length: 54976, dtype: object

In [45]:
rank_mean = [] # 도서 평균 평점
rank_percentage = [] # 평균 percentage
review_title = [] # 리뷰 제목
review_id = [] # 리뷰 id
review_rank = [] # 리뷰 평점
review_text = [] # 리뷰 
isbn_idx = []


for isbn in isbn13[37601:38000]: # 페이지 입력
    url = "http://www.yes24.com/product/search?query={}".format(isbn)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('a.gd_name')
    goods = []
    for link in book_links:
        a= link['href'].split('/')[3] 
        goods.append(a)
    for item in goods:
        del(goods[1:])
        driver.get('http://www.yes24.com'+'/Product/Goods/{}'.format(item)) # 상품 페이지
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            mean = soup.select('div.gd_rating > em.yes_b')[0].text
            rank_mean.append([mean]) # 도서 평균 평점
        except:
            mean = []
            rank_mean.append(mean) # 도서 평균 평점
        rank_p = []
        try:
            for k in range(5):
                rank_ = soup.select('span.barGraphBase')[k].text # 평점 percentage
                rank_p.append(rank_)
            rank_percentage.append(rank_p)
        except:
            rank_ = []
            rank_percentage.append(rank_)
        try:
            review_t = []  # 각각의 책의 리뷰제목 리스트
            review_i = []
            review_r = []
            review_te = []
            for page in range(1,3): # 리뷰 페이지 2개
                driver.get('http://www.yes24.com/Product/communityModules/GoodsReviewList/{}?Type=ALL&_=1648001707174&Sort=2&PageNumber={}'.format(item, page))
                time.sleep(1)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                blog_link = []
                for i in range(5): # 리뷰 한 페이지에 있는 5개 리뷰 가져오기
                    try:
                        title = soup.select('span.review_tit')[i].text.strip() # 리뷰 제목
                        review_t.append(title)
                        r_id = soup.select('a.lnk_id')[i].text # 리뷰 id
                        review_i.append(r_id)
                        rank = soup.select('span.review_rating')[i].text.split('\n')[1][-2:] # 리뷰 평점
                        review_r.append(rank)
                        blog_l = soup.select('div.review_lnk a')[i]['href'] # 리뷰 블로그 링크
                        blog_link.append(blog_l)
                    except:
                        pass
                for j in range(5): # 리뷰 한 페이지에 있는 5개 블로그 내용 가져오기
                        driver.get(blog_link[j]) # 리뷰 블로그 링크 들어가기
                        time.sleep(1)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        text = soup.select('div.blogContArea')[0].text # 리뷰 내용 
                        review_te.append(text)
        except:
            pass
        review_title.append(review_t)
        review_id.append(review_i)
        review_rank.append(review_r)
        review_text.append(review_te)
        isbn_idx.append(isbn)

In [46]:
col = ['isbn','rank_mean','rank_percentage','review_title','review_id','review_rank','review_text']
user = pd.DataFrame([isbn_idx,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text]).T
user.columns = col
user

,isbn,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,9788984995000,[],[],[],[],[],[]
1,9788932016450,[],[],[],[],[],[]
2,9788932015835,[2.0],"[0%, 0%, 0%, 0%, 100%]",[번역의 아쉬움...],[w*******i],[1점],[\n이 책은 정말 정말 추천하고 싶지 않은 책이다.\n \n아니면..나는 이 책이...
3,9788982816567,[8.5],"[50%, 50%, 0%, 0%, 0%]","[상처받는게 두려운가, 일상의 갈등을 포착한 한국의 여자 레이몬드 카버....]","[지*, k******i]","[5점, 3점]",[\n난 겁이 많다. 여기서 겁이란 호러영화를 보거나 자이로스윙같은걸 탈때의 두려움...
4,9788974282141,[8.0],"[0%, 100%, 0%, 0%, 0%]",[복수의 방랑자 그리고 사랑],[j******3],[4점],"[\n빅토리아 시대의 런던, 도도하고 아름다운 공작의 외동딸 레이디 사라 세인트 제..."
...,...,...,...,...,...,...,...
391,9788971155028,[],[],[],[],[],[]
392,9788961461016,[7.0],"[17%, 33%, 42%, 8%, 0%]","[여성과 여왕사이에서, 통일신라의 중심에 선덕여왕이 있다..., 아름답고 용감한 선...","[고*미, a******m, 은**미, 공**마, 예***빠, 하*롤, 해*고, 땡...","[5점, 5점, 5점, 5점, 4점, 4점, 4점, 4점, 4점, 4점]",[\n조선왕조 이전에는 여성의 지위가 얽매이거나 낮지는 않았던 것 같다. 그래도 그...
393,9788960210103,[],[],[],[],[],[]
394,9788908062221,[],[],[],[],[],[]


In [48]:
user.to_pickle('user_37601-37999.pkl', protocol = 4)

In [47]:
for i in range(37601,38000):
    if isbn13[i] == '9788937420252':
        print(i)

37999


In [49]:
user1 = pd.read_pickle('user_37001-37300.pkl')
user2 = pd.read_pickle('user_37301-37600.pkl')
user3 = pd.read_pickle('user_37601-37999.pkl')

In [50]:
df = pd.concat([user1, user2,user3],ignore_index=True)
df.to_pickle('user_37001-37999.pkl', protocol = 4)

In [44]:
6999 - 4474

2525

In [43]:
len(df)

2531

In [132]:
df['isbn'].duplicated().sum()

33